### Simple test notebook
This notebook is a simple test of the functionality of the SOCS method with a very small dataset, to ensure that the software works properly.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io as io
from scipy import stats
import scanpy as sc
import anndata as ad
import squidpy as sq
import copy
import seaborn as sns
import pickle
import math
import sklearn
import torch
from unbalancedgw_f.vanilla_ugw_solver import log_ugw_sinkhorn_f

//broad/clearylab/Users/Peter/anaconda3/envs/myEnv/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


We'll load in a very small sample from the two timepoints in the MERFISH mouse ovary datasets.

In [123]:
adata_1 = sc.read_h5ad('adata_1_follicle.h5ad')
adata_2 = sc.read_h5ad('adata_2_follicle.h5ad')

We'll obtain the dimensionally-reduced count tables $G^{(p)}_1$ and $G^{(p)}_2$, and the spatial location vectors $x_1$, $x_2$, $y_1$, and $y_2$, as defined in Eq. FILL IN in our manuscript.

In [124]:
adata_concat= ad.concat([adata_1,adata_2])
sc.pp.pca(adata_concat, random_state= 0,n_comps=30)
adata_concat_1 = adata_concat[0:adata_1.shape[0],:]
adata_concat_2 = adata_concat[adata_1.shape[0]:,:]
G_p_1 = adata_concat_1.obsm['X_pca']
G_p_2 = adata_concat_2.obsm['X_pca']
x_1 = adata_concat_1.obsm['spatial'][:,0]
y_1 = adata_concat_1.obsm['spatial'][:,1]
x_2 = adata_concat_2.obsm['spatial'][:,0]
y_2 = adata_concat_2.obsm['spatial'][:,1]
xy_1 = np.stack([x_1,y_1],axis=1)
xy_2 = np.stack([x_2,y_2],axis=1)

//broad/clearylab/Users/Peter/anaconda3/envs/myEnv/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12030. The TBB threading layer is disabled.
  warnings.warn(problem)


This cell computes the matrices $D_g$, $D_1$, and $D_2$, along with the normalization factors $f_1$ and $f_2$, as defined in Eq. FILL IN in our manuscript

In [125]:
D_g = np.ascontiguousarray(sklearn.metrics.pairwise.pairwise_distances(G_p_1,Y=G_p_2,metric='euclidean',n_jobs=1))
D_1 = np.ascontiguousarray(sklearn.metrics.pairwise.pairwise_distances(xy_1,Y=xy_1,metric='euclidean',n_jobs=1))
D_2 = np.ascontiguousarray(sklearn.metrics.pairwise.pairwise_distances(xy_2,Y=xy_2,metric='euclidean',n_jobs=1))
f1 = (np.max(D_1)/np.max(D_g))**2
f2 = (np.max(D_2)/np.max(D_g))**2
nCells_1 = adata_1.shape[0]
nCells_2 = adata_2.shape[0]
p1 = np.ones([nCells_1,])/nCells_1
p2 = np.ones([nCells_2,])/nCells_2
D_g_torch = torch.tensor(D_g,dtype=torch.float64)
D_1_torch = torch.tensor(D_1,dtype=torch.float64)
D_2_torch = torch.tensor(D_2,dtype=torch.float64)
p1_torch = torch.tensor(p1,dtype=torch.float64)
p2_torch = torch.tensor(p2,dtype=torch.float64)

Finally, we'll use the SOCS algorithm to estimate the trajectory mapping $T$ between the samples at $t_1$ and $t_2$.

In [186]:
alpha = 0.5
eps = 2e-4
rho1 = 5000.0
rho2 = 5000.0
T_torch, gamma = log_ugw_sinkhorn_f(p1_torch, D_1_torch/f1, p2_torch, D_2_torch/f2, D_g_torch, alpha, init=None, eps=eps,
    rho=rho1, rho2=rho2,
    nits_plan=30, tol_plan=1e-30,
    nits_sinkhorn=10, tol_sinkhorn=1e-9,
    two_outputs=False,print_per_iter=10,alt=0)

T = T_socs_torch.numpy()

0
10
20
